<a href="https://colab.research.google.com/github/Aruna-Jayarajan/Satelite_Network/blob/main/GNN_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import os

# Add `src/` directory to module search path
sys.path.append(os.path.join(os.getcwd(), 'src'))

# Now import project modules
from dataloader import SatelliteDataset, NUM_GATEWAYS
from model import Stage2GNN
from train import train_model_with_mse
from utils import plot_metrics


In [ ]:
from torch_geometric.loader import DataLoader as PyGDataLoader

DATA_FOLDER = r"C:\Users\aruna\Desktop\MS Thesis\Real Data\Final folder real data"
file_list = sorted([os.path.join(DATA_FOLDER, f) for f in os.listdir(DATA_FOLDER) if f.endswith('.csv')])[:10]

train_size = int(0.8 * len(file_list))
train_files, val_files = file_list[:train_size], file_list[train_size:]

train_dataset = SatelliteDataset(train_files)
val_dataset = SatelliteDataset(val_files)

train_loader = PyGDataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = PyGDataLoader(val_dataset, batch_size=16, shuffle=False)

print(f"Loaded {len(train_files)} training files and {len(val_files)} validation files.")


Loaded 8 training files and 2 validation files.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tensorflow.keras.models import load_model

STAGE1_MODEL_PATH = 'stage_1_visible_model.h5'
stage1_model = load_model(STAGE1_MODEL_PATH)

input_dim = 3 + NUM_GATEWAYS * 3
gnn_model = Stage2GNN(input_dim=input_dim, sat_feature_dim=111, neighbor_feature_dim=NUM_GATEWAYS,
                      hidden_dim=256, output_dim=NUM_GATEWAYS)

optimizer_gnn = optim.Adam(gnn_model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [ ]:
results = train_model_with_mse(
    gnn_model=gnn_model,
    stage1_model=stage1_model,
    train_loader=train_loader,
    val_loader=val_loader,
    loss_fn=loss_fn,
    optimizer_gnn=optimizer_gnn,
    num_epochs=10,
    rounds=15
)

(train_losses, val_losses,
 train_top1_acc, train_top3_acc, train_top5_acc,
 val_top1_acc, val_top3_acc, val_top5_acc) = results


Epoch 2, Train Loss: 3.9795, Val Loss: 3.9470


KeyboardInterrupt: 

In [ ]:
plot_metrics(train_losses, val_losses,
             train_top1_acc, train_top3_acc, train_top5_acc,
             val_top1_acc, val_top3_acc, val_top5_acc)

# Save model
torch.save(gnn_model.state_dict(), 'stage2_loop_gnn_model.pth')
print("Model saved.")
